In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

sys.path.append(
    "/Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/"
)

from backend.data_preprocessing.pipeline import run_pipeline

In [2]:
cart_df, order_df, inventory_df, retail_df = run_pipeline()
products_df = pd.read_csv("/Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/data/processed/products.csv")

[INFO] Loading datasets...
[INFO] Successfully loaded /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/data/processed/cart.csv with 3000 rows
[INFO] Successfully loaded /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/data/processed/order.csv with 1468 rows
[INFO] Successfully loaded /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/data/processed/inventory.csv with 519 rows
[INFO] Successfully loaded /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/data/processed/retail.csv with 1740 rows
[INFO] Preprocessing Cart data...
[INFO] Cleaned 'total_amount' from currency format. Valid non-null: 3000/3000.
[INFO] Removed 1100 abandoned cart rows.
[INFO] Removed 12 duplicate rows based on ['order_id', 'order_date'].
[INFO] Preprocessing Order data...
[INFO] Removed 81 rows before 2021-03-31 00:00:00.
[INF

In [3]:
BASE_DIR = "/Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output"
EDA_PLOTS_DIR = os.path.join(BASE_DIR, "fig")
os.makedirs(EDA_PLOTS_DIR, exist_ok=True)


def basic_info(df: pd.DataFrame, df_name: str) -> None:
    """
    Print basic information about a DataFrame: shape, columns, missing values,
    and descriptive stats.
    """
    print(f"\n=== {df_name.upper()} DataFrame Info ===")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")

    # Check for missing values
    missing_count = df.isna().sum()
    print(f"\nMissing Values:\n{missing_count[missing_count > 0]}")

    # Display dtypes
    print("\nData Types:")
    print(df.dtypes)

    # Basic numerical summary
    print("\nBasic Descriptive Statistics (numeric columns):")
    print(df.describe(include=[float, int]))


def plot_histograms(df: pd.DataFrame, numeric_cols: list, df_name: str) -> None:
    """
    Generate histograms for a list of numeric columns in the given DataFrame.
    """
    for col in numeric_cols:
        if col not in df.columns:
            continue
        plt.figure(figsize=(6, 4))
        sns.histplot(df[col], kde=True, bins=30)
        plt.title(f"{df_name}: Distribution of {col}")
        plt.xlabel(col)
        plt.tight_layout()

        # Save the figure to EDA_PLOTS_DIR
        plot_path = os.path.join(EDA_PLOTS_DIR, f"{df_name}_hist_{col}.png")
        plt.savefig(plot_path)
        plt.close()  # Close the plot to free memory
        print(f"[INFO] Saved histogram for {col} -> {plot_path}")


def plot_correlation_heatmap(df: pd.DataFrame, df_name: str) -> None:
    """
    Generate and save a correlation heatmap for the numeric columns of the DataFrame.
    """
    if df.select_dtypes(include=[float, int]).empty:
        print(f"[WARN] No numeric columns found in {df_name} to compute correlation.")
        return

    corr = df.select_dtypes(include=[float, int]).corr()
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", square=True)
    plt.title(f"{df_name}: Correlation Heatmap")
    plt.tight_layout()

    plot_path = os.path.join(EDA_PLOTS_DIR, f"{df_name}_corr_heatmap.png")
    plt.savefig(plot_path)
    plt.close()
    print(f"[INFO] Saved correlation heatmap -> {plot_path}")


# 2. Main EDA Flow
# ------------------------------------------------------------------------------
def main():
    """
    Main function that orchestrates the EDA steps:
    1. Load cleaned data (cart, order, inventory, retail).
    2. Print basic info for each DataFrame.
    3. Plot histograms and correlation heatmaps (where numeric columns exist).
    """

    # Basic info
    basic_info(cart_df, "cart")
    basic_info(order_df, "order")
    basic_info(inventory_df, "inventory")
    basic_info(retail_df, "retail")

    # Plot histograms for certain numeric columns
    # Adjust the list of numeric columns based on your dataset.
    cart_numeric = ["total"]  # Example
    order_numeric = ["total"]  # Example
    inventory_numeric = ["qty", "factory_price", "retail", "retail_us"]
    retail_numeric = ["pv_ttc", "ca_ttc"]

    plot_histograms(cart_df, cart_numeric, "cart")
    plot_histograms(order_df, order_numeric, "order")
    plot_histograms(inventory_df, inventory_numeric, "inventory")
    plot_histograms(retail_df, retail_numeric, "retail")

    # Plot correlation heatmaps
    plot_correlation_heatmap(cart_df, "cart")
    plot_correlation_heatmap(order_df, "order")
    plot_correlation_heatmap(inventory_df, "inventory")
    plot_correlation_heatmap(retail_df, "retail")

    print(
        "\n[INFO] EDA analysis complete. Check console output and eda_plots directory for results."
    )


In [4]:
main()


=== CART DataFrame Info ===
Shape: (2985, 6)
Columns: ['id', 'order_id', 'customer', 'total_amount', 'carrier', 'order_date']

Missing Values:
customer    2267
carrier     2594
dtype: int64

Data Types:
id                       int64
order_id                object
customer                object
total_amount           float64
carrier                 object
order_date      datetime64[ns]
dtype: object

Basic Descriptive Statistics (numeric columns):
                 id  total_amount
count   2985.000000   2985.000000
mean    8652.915243    918.126298
std      912.887523   2274.325165
min     6921.000000      0.000000
25%     7928.000000      0.000000
50%     8680.000000    285.000000
75%     9436.000000    880.000000
max    10187.000000  44844.000000

=== ORDER DataFrame Info ===
Shape: (1360, 9)
Columns: ['id', 'référence', 'nouveau_client', 'livraison', 'client', 'total', 'paiement', 'état', 'date']

Missing Values:
client    3
dtype: int64

Data Types:
id                         int64

/Users/benpfeffer/Anaconda_FOLDER/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/benpfeffer/Anaconda_FOLDER/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


[INFO] Saved histogram for total -> /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output/fig/order_hist_total.png
[INFO] Saved histogram for qty -> /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output/fig/inventory_hist_qty.png


/Users/benpfeffer/Anaconda_FOLDER/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


[INFO] Saved histogram for factory_price -> /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output/fig/inventory_hist_factory_price.png


...

[INFO] Saved histogram for retail -> /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output/fig/inventory_hist_retail.png


/Users/benpfeffer/Anaconda_FOLDER/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


[INFO] Saved histogram for retail_us -> /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output/fig/inventory_hist_retail_us.png
[INFO] Saved histogram for pv_ttc -> /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output/fig/retail_hist_pv_ttc.png


/Users/benpfeffer/Anaconda_FOLDER/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


[INFO] Saved correlation heatmap -> /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output/fig/cart_corr_heatmap.png
[INFO] Saved correlation heatmap -> /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output/fig/order_corr_heatmap.png
[INFO] Saved correlation heatmap -> /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output/fig/inventory_corr_heatmap.png
[INFO] Saved correlation heatmap -> /Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/output/fig/retail_corr_heatmap.png

[INFO] EDA analysis complete. Check console output and eda_plots directory for results.


In [15]:
cart = pd.read_csv("/Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/data/cleaned/cart.csv")
inventory = pd.read_csv("/Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/data/cleaned/inventory.csv")
order = pd.read_csv("/Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/data/cleaned/order.csv")
retail = pd.read_csv("/Users/benpfeffer/Library/Mobile Documents/com~apple~CloudDocs/projects-portfolio-main/5-octobre/data/cleaned/retail.csv")

In [14]:

inventory["cost"] = inventory["factory_price"] * inventory["qty"]

total_revenue = retail["pv_ttc"].sum()
total_cost = inventory["cost"].sum()
if total_revenue > 0:
    gross_profit_margin = ((total_revenue - total_cost) / total_revenue) * 100
else:
    gross_profit_margin = 0

gross_profit_margin

90.71168140590608

In [11]:
if 'order_id' in cart.columns:
    cart.drop(cart.index[cart.order_id == 'Panier abandonné'], inplace=True)

cart.drop_duplicates(inplace=True)

,id,référence,nouveau_client,livraison,client,total,paiement,état,date
0,468,EOCSEVOVW,1,Lituanie,K. Kybartaite,205.0,Card via Stripe,Livré,2022-04-29 12:52:00
1,467,YKNOEEZEX,1,Suisse,F. zanni,1542.0,Card via Stripe,Livré,2022-04-28 21:22:45
2,466,LHCRAEYPL,1,Belgique,O. Voortman,780.0,Transfert bancaire,Livré,2022-04-26 13:57:29
3,465,IVTTHQOYH,1,France,R. Jeanjean,480.0,Card via Stripe,Livré,2022-04-25 20:52:39
4,464,FQDTAFHIK,1,Suisse,A. Kiialainen,208.0,Card via Stripe,Livré,2022-04-20 23:22:20


In [30]:
inventory_df = inventory.copy()
inventory_df["total_cost"] = inventory_df["factory_price"] * inventory_df["qty"]

inventory_df['margin_eur'] =  inventory_df['retail'] - inventory_df['total_cost']
inventory_df['margin_eur_pct'] = inventory_df['margin_eur'] / inventory_df['retail']


inventory_df['margin_us'] =  inventory_df['retail_us'] - inventory_df['total_cost']
inventory_df['margin_us_pct'] = inventory_df['margin_us'] / inventory_df['retail_us']

inventory_df.head()
print("AVG MARGIN EUR vs US:")
print("EUR: ", inventory_df["margin_eur"].mean())
print("USD: ", inventory_df["margin_us"].mean())


AVG MARGIN EUR vs US:
EUR:  815.035626204239
USD:  1260.2706936416184


In [39]:
inventory_df.sort_values(by="margin_eur", ascending=False).head(10)
inventory_df.sort_values(by="margin_us_pct", ascending=True).head(15)


,id,sfa,lib,ean,qty,factory_price,retail,retail_us,total_cost,margin_eur,margin_eur_pct,margin_us,margin_us_pct
12,2714,Emballage,BOITE LARGE black gold-black gold,3666023025948,7,0.80,0.0,0.0,5.60,-5.60,-inf,-5.60,-inf
364,8767,Coffret,Coffret gainé en édition limit Rouge Burgundy-...,3666023083016,2,26.00,120.0,140.0,52.00,68.00,0.566667,88.00,0.628571
363,8766,Coffret,Coffret gainé en édition limit Rouge Burgundy-...,3666023083009,2,26.00,120.0,140.0,52.00,68.00,0.566667,88.00,0.628571
0,918,Creoles,LARA AR - PRESTA JAMES Argent doré,3666023008545,4,22.33,180.0,270.0,89.32,90.68,0.503778,180.68,0.669185
515,9737,Boucles d'oreilles,KHANO L ** Or 14k,3666023092544,2,507.97,2135.0,3960.0,1015.94,1119.06,0.524150,2944.06,0.743449
20,3792,Bague,JAY XL Or 14k 53,3666023035695,3,97.48,790.0,1140.0,292.44,497.56,0.629823,847.56,0.743474
218,7318,Collier,FELIX R.* Or 14k - rubis,3666023069072,3,75.72,620.0,900.0,227.16,392.84,0.633613,672.84,0.747600
504,9713,Bague,Nine Sun ** Or 14k - Diamant 53,3666023092308,2,123.06,650.0,1010.0,246.12,403.88,0.621354,763.88,0.756317
503,9712,Bague,Nine Sun ** Or 14k - Diamant 51,3666023092292,2,123.06,650.0,1010.0,246.12,403.88,0.621354,763.88,0.756317
324,8178,Bague,GLORIA Or 14k - Diamant 53,3666023077312,2,142.72,830.0,1200.0,285.44,544.56,0.656096,914.56,0.762133
